In [ ]:
import asyncio
%pip install ballsort
from ballsort.ballsort_display_utils import open_bs_window
open_bs_window()

In [ ]:
from ball_control import BallControl
from state_update_model import StateBall, StatePosition

async def move_ball(bc: BallControl, src: StatePosition, dest: StatePosition, claw_index: int = 0):
    rel_x = src.x - bc.get_position(claw_index=claw_index).x
    rel_y = src.y - bc.get_position(claw_index=claw_index).y
    await asyncio.gather(
        bc.move_horizontally(rel_x, claw_index=claw_index),
        bc.move_vertically(rel_y, claw_index=claw_index),
        bc.open_claw(claw_index=claw_index))
    await bc.close_claw(claw_index=claw_index)
    
    rel_x = dest.x - bc.get_position(claw_index=claw_index).x
    rel_y = dest.y - bc.get_position(claw_index=claw_index).y
    await asyncio.gather(
        bc.move_horizontally(rel_x, claw_index=claw_index),
        bc.move_vertically(rel_y, claw_index=claw_index))
    await bc.open_claw(claw_index=claw_index)

async def move_ball_by_column(bc: BallControl, src_x: int, dest_x: int, claw_index: int = 0):
    src_column_top_occupied_y = min([ball.pos.y for ball in bc.get_state().balls if ball.pos.x == src_x],default=bc.get_state().max_y)
    dest_column_top_vacant_y = min([ball.pos.y for ball in bc.get_state().balls if ball.pos.x == dest_x],default=bc.get_state().max_y + 1) - 1
    await move_ball(bc=bc, src=StatePosition(x=src_x, y=src_column_top_occupied_y), dest=StatePosition(x=dest_x, y=dest_column_top_vacant_y), claw_index=claw_index)

In [ ]:
from control_factory import get_control_sim
from ch10_scenario import Ch10Scenario

In [ ]:
async def challenge10_solution():
    bc = get_control_sim(delay_multiplier=0.2)
    await bc.set_scenario(Ch10Scenario(seed=None))
    
    reveal_spot_x = bc.get_state().max_x
    reveal_spot_y = bc.get_state().max_y

    # reveal each ball's value by moving it to the revealer spot. Then move it to bucket column.
    for _ in range(len(bc.get_state().balls)):
        await move_ball_by_column(bc=bc, src_x=0, dest_x=reveal_spot_x) # reveal
        revealed_value = next((ball.value for ball in bc.get_state().balls if ball.pos == StatePosition(x=reveal_spot_x, y=reveal_spot_y)), None)
        assert(revealed_value)
        await move_ball_by_column(bc=bc, src_x=reveal_spot_x, dest_x=revealed_value) # move to column corresponding to value

    # Finally move all buckets to column 0 in the correct order
    for x in range(3, 0, -1):
        nof_balls_in_column = len([ball for ball in bc.get_state().balls if ball.pos.x == x])
        for _ in range(nof_balls_in_column):
            await move_ball_by_column(bc=bc, src_x=x, dest_x=0)

    assert bc.get_state().goal_accomplished

In [ ]:
await challenge10_solution()